In [582]:
import gensim
from gensim import corpora
from gensim import similarities
from gensim import models
from gensim.models import CoherenceModel

import preprocess

# Load LDA Model

In [583]:
lda_disk=gensim.models.ldamodel.LdaModel.load("Model/finalmodel_5Topics")

In [584]:
id2word = corpora.Dictionary.load('Model/finalmodel_Dictionary')

# Extract & Preprocess Test Data

In [585]:
test_corpus_1974 = preprocess.load_corpus('Data/Test/Chapters/5827')

test_ids = test_corpus_1974.fileids()
chapters_name = [id.replace('.txt','') for id in test_ids]

test_docs_1974 = preprocess.corpus2docs(test_corpus_1974)

In [586]:
def make_bigrams(bigram_mod, texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(bigram_mod, trigram_mod, texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [587]:
bigram = gensim.models.Phrases(test_docs_1974, min_count=5, threshold=50) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[test_docs_1974], threshold=50)  

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

docs_bigrams = make_bigrams(bigram_mod, test_docs_1974)
data_bigrams_trigrams = make_trigrams(bigram_mod, trigram_mod, docs_bigrams)

In [588]:
test_vecs_1974 = preprocess.docs2vecs(data_bigrams_trigrams, id2word)
similarity = similarities.MatrixSimilarity(lda_disk[test_vecs_1974])

In [589]:
for i in range(0,len(test_vecs_1974)):
    vector = lda_disk[test_vecs_1974[i]]
    sim_topic = max(vector,key=lambda item:item[1])
    print("test doc" + str(i) + ": " + str(vector))
    print("Closest Topic: Topic " + str(sim_topic[0]))

test doc0: [(0, 0.2092944), (1, 0.04010017), (2, 0.66173494), (3, 0.012143663), (4, 0.07672681)]
Closest Topic: Topic 2
test doc1: [(0, 0.06492638), (1, 0.074570395), (2, 0.5798887), (4, 0.27246475)]
Closest Topic: Topic 2
test doc2: [(0, 0.010751086), (1, 0.19740404), (2, 0.56492525), (4, 0.22044188)]
Closest Topic: Topic 2
test doc3: [(0, 0.07416345), (1, 0.09154141), (2, 0.39488274), (4, 0.43103626)]
Closest Topic: Topic 4
test doc4: [(0, 0.018251669), (1, 0.10464484), (2, 0.48008782), (3, 0.025361067), (4, 0.3716546)]
Closest Topic: Topic 2
test doc5: [(0, 0.08246775), (1, 0.09601282), (2, 0.5152149), (3, 0.01635788), (4, 0.28994665)]
Closest Topic: Topic 2
test doc6: [(0, 0.13596003), (1, 0.13214675), (2, 0.25238892), (3, 0.041432526), (4, 0.43807176)]
Closest Topic: Topic 4
test doc7: [(0, 0.09320552), (1, 0.11042821), (2, 0.50177413), (4, 0.29451278)]
Closest Topic: Topic 2
test doc8: [(0, 0.054623727), (1, 0.012102318), (2, 0.3786071), (3, 0.012749717), (4, 0.54191715)]
Closest

In [590]:
topic_word = lda_disk.show_topic(0, topn=len(id2word))
topic_word[0:20]

[('conception', 0.03775606),
 ('zarathustra', 0.028077127),
 ('thou', 0.024449555),
 ('hath', 0.018459603),
 ('pure', 0.016960025),
 ('phenomenon', 0.015507126),
 ('thee', 0.0153419515),
 ('unto', 0.01463752),
 ('cognition', 0.013255156),
 ('intuition', 0.013230192),
 ('empirical', 0.010757555),
 ('social', 0.010513507),
 ('activity', 0.010492589),
 ('education', 0.0104921805),
 ('representation', 0.009958939),
 ('priori', 0.009342978),
 ('verily', 0.009038341),
 ('doth', 0.008945876),
 ('space', 0.008808358),
 ('transcendental', 0.008680221)]

## Model 1 - Using Similarity Score

### Top Keywords

In [591]:
chap_num = 0
vector = lda_disk[test_vecs_1974[chap_num]]
sim_topic = max(vector,key=lambda item:item[1])
top_topic = sim_topic[0]
top_topic
topic_word = lda_disk.show_topic(top_topic, topn=len(id2word))
# topic_word

In [592]:
selected_words = [id2word[i[0]] for i in test_vecs_1974[chap_num]]
selected_words[0:10]

['able',
 'acquire',
 'action',
 'adopt',
 'advance',
 'afford',
 'affords',
 'agree',
 'almost',
 'along']

In [593]:
key_words = []

keyword_type = 'bigrams'
for word in topic_word:
    if(keyword_type == 'unigrams'):
        if (len(key_words) < 5) & (word [0] in selected_words):
            key_words.append(word)
    else:
        if ('_' in word[0]) & (len(key_words) < 5) & (word [0] in selected_words):
            key_words.append(word)

key_words

[]

### Recommender

In [594]:
chapters_name

['CHAPTER_III__THE_NATURE_OF_MATTER',
 'CHAPTER_II__THE_EXISTENCE_OF_MATTER',
 'CHAPTER_IV__IDEALISM',
 'CHAPTER_IX__THE_WORLD_OF_UNIVERSALS',
 'CHAPTER_I__APPEARANCE_AND_REALITY',
 'CHAPTER_VIII__HOW__A_PRIORI__KNOWLEDGE_IS_POSSIBLE',
 'CHAPTER_VII__ON_OUR_KNOWLEDGE_OF_GENERAL_PRINCIPLES',
 'CHAPTER_VI__ON_INDUCTION',
 'CHAPTER_V__KNOWLEDGE_BY_ACQUAINTANCE_AND_KNOWLEDGE_BY_DESCRIPTION',
 'CHAPTER_XIII__KNOWLEDGE,_ERROR,_AND_PROBABLE_OPINION',
 'CHAPTER_XII__TRUTH_AND_FALSEHOOD',
 'CHAPTER_XIV__THE_LIMITS_OF_PHILOSOPHICAL_KNOWLEDGE',
 'CHAPTER_XI__ON_INTUITIVE_KNOWLEDGE',
 'CHAPTER_X__ON_OUR_KNOWLEDGE_OF_UNIVERSALS']

In [595]:
chap_num = 8

chosen_chapter = chapters_name[chap_num]
recommendation_scores = []

for i in range(0,len(test_vecs_1974)):
    vector = lda_disk[test_vecs_1974[i]]
    sim_topic = max(vector,key=lambda item:item[1])
    
    if(i == chap_num):
        sims = similarity[vector]
        sims = list(enumerate(sims))
        for sim in sims:
            chapter_num = sim[0]
            recommendation_score = [chapters_name[chapter_num], sim[1]]
            recommendation_scores.append(recommendation_score)
        
recommendation_scores = sorted(recommendation_scores, key=lambda x: x[1], reverse=True)     
recommendation = []


print('Top 3 Recommended Chapters:')
for i in range(1,4):
    recommendation.append((recommendation_scores[i][0], recommendation_scores[i][1]))
    print(f'{i}. {str(recommendation_scores[i][0])}({str(recommendation_scores[i][1])})')

Top 3 Recommended Chapters:
1. CHAPTER_X__ON_OUR_KNOWLEDGE_OF_UNIVERSALS(0.98627436)
2. CHAPTER_IX__THE_WORLD_OF_UNIVERSALS(0.9811799)
3. CHAPTER_XIII__KNOWLEDGE,_ERROR,_AND_PROBABLE_OPINION(0.9807602)


## Model 2 - Using Distribution of Topics

In [596]:
chapters_name

['CHAPTER_III__THE_NATURE_OF_MATTER',
 'CHAPTER_II__THE_EXISTENCE_OF_MATTER',
 'CHAPTER_IV__IDEALISM',
 'CHAPTER_IX__THE_WORLD_OF_UNIVERSALS',
 'CHAPTER_I__APPEARANCE_AND_REALITY',
 'CHAPTER_VIII__HOW__A_PRIORI__KNOWLEDGE_IS_POSSIBLE',
 'CHAPTER_VII__ON_OUR_KNOWLEDGE_OF_GENERAL_PRINCIPLES',
 'CHAPTER_VI__ON_INDUCTION',
 'CHAPTER_V__KNOWLEDGE_BY_ACQUAINTANCE_AND_KNOWLEDGE_BY_DESCRIPTION',
 'CHAPTER_XIII__KNOWLEDGE,_ERROR,_AND_PROBABLE_OPINION',
 'CHAPTER_XII__TRUTH_AND_FALSEHOOD',
 'CHAPTER_XIV__THE_LIMITS_OF_PHILOSOPHICAL_KNOWLEDGE',
 'CHAPTER_XI__ON_INTUITIVE_KNOWLEDGE',
 'CHAPTER_X__ON_OUR_KNOWLEDGE_OF_UNIVERSALS']

In [597]:
# if first digit in tuple matches, store doc
# reco_docs -- dont remove user input
# - get index of user input within reco_docs
# - compare difference between first topic probability and store as list; e.g. {4: 0.002,...}
# - take top 5 minimum difference

j = 0
vector_selected = lda_disk[test_vecs_1974[j]]
vector_selected.sort(key=lambda x: x[1], reverse=True)
vector_selected

reco_docs = {}

for i in range(0,len(test_vecs_1974)):
    vector = lda_disk[test_vecs_1974[i]]
    vector.sort(key=lambda x: x[1], reverse=True)
    
    if (vector[0][0]==vector_selected[0][0]) & (vector[1][0] == vector_selected[1][0]):
        reco_docs[i] = vector
        # print(vector[0][1])

# print(reco_docs)
input_topic = reco_docs[j]

# Remove user input
if j in reco_docs.keys():
    reco_docs.pop(j)

# if there's no 2 same topics
if reco_docs == {}:
    for i in range(0,len(test_vecs_1974)):
        vector = lda_disk[test_vecs_1974[i]]
        vector.sort(key=lambda x: x[1], reverse=True)
        
        if (vector[0][0]==vector_selected[0][0]):
            reco_docs[i] = vector

    # Remove user input
    if j in reco_docs.keys():
        reco_docs.pop(j)

diff = ""
diff_dict = {}
for x, y in reco_docs.items():
    diff = abs(input_topic[0][1] - reco_docs[x][0][1])
    diff_dict[x] = diff

diff_dict = sorted(diff_dict.items(), key=lambda x:x[1], reverse=False)
diff_dict = dict(diff_dict)
# print(diff_dict)

lst = list(diff_dict.keys())
# print(lst[0:5])

print('Top 3 Recommended Chapters:')
for i in range(0,len(lst)):
    if i < 3:
        print(f'{i+1}. {chapters_name[lst[i]]}')

Top 3 Recommended Chapters:
1. CHAPTER_II__THE_EXISTENCE_OF_MATTER
2. CHAPTER_IV__IDEALISM
3. CHAPTER_XII__TRUTH_AND_FALSEHOOD
